In [3]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [13]:
from tqdm import tqdm
import os
import random

In [5]:
apikey = ''

In [6]:
import requests

def curl(value):
    headers = {
        'Content-Type': 'application/json',
        'api-key': apikey,
    }

    params = {
        'api-version': '2023-03-15-preview',
    }

    json_data = {
        'messages': [
            {'role': 'system', 'content':'answer only, no need to explain'},
            {
                'role': 'user',
                'content': value,
            },
        ],
        'max_tokens': 100,
        'temperature': 0,
    }

    response = requests.post(
        'https://nous.openai.azure.com/openai/deployments/nous/chat/completions',
        params=params,
        headers=headers,
        json=json_data,
        timeout = 120,
    )
    
    return response

In [7]:
with open('BM-A-pt3') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    objektif = t.split('objektif: ')[1].split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'objektif': objektif,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [12]:
arange = set(range(len(questions)))

In [28]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
objektif: {row['objektif']}
soalan: {row['soalan']}
    """
    return prompt.strip()

In [32]:
i = 0
shots = random.sample(arange - {i}, 1)
prompts = []
for no, s in enumerate(shots):
    prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts)
print(prompt)

Contoh soalan 1
objektif: Pilih ayat-ayat yang betul.
soalan: I. Encik Ayob gembira apabila anaknya bersuamikan dengan lelaki pilihannya.
II. "Patik menjunjung kasih atas perkenan Tuanku menyempumakan upacara ini," kata pengacara majiis.
III. Pelajar saling berlukar pendapat berkaitan isu yang diberi oleh guru.
IV. Jalan raya di Malaysia adalah antara yang paling baik di rantau Asia.

A. I dan II sahaja
B. III dan IV sahaja
C. I, II, dan III sahaja
D. II, III, dan IV sahaja
jawapan: D

objektif: Lengkapkan ayat-ayat yang berikut dengan memilih jawapan yang paling sesuai.
soalan: Para ___ tanah air telah dihantar ke negara Jepun untuk membantu mangsa gempa bumi dan tsunami.
A. hartawan
B. dermawan
C. bangsawan
D. sukarelawan


In [33]:
r = curl(prompt)

In [34]:
r.json()['choices'][0]['message']['content']

'D. sukarelawan'

In [39]:
for i in tqdm(range(len(questions))):
    shots = random.sample(arange - {i}, 1)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    try:
        r = curl(prompt)
        answer = r.json()['choices'][0]['message']['content']
        questions[i]['output'] = answer
    except:
        pass

100%|███████████████████████████████████████████| 54/54 [01:08<00:00,  1.27s/it]


In [40]:
import json

with open('output-1shot.json', 'w') as fopen:
    json.dump(questions, fopen)

In [41]:
filtered = [q for q in questions if 'output' in q]
len(filtered)

52

In [43]:
correct = 0
for q in filtered:
    correct += q['output'].split('.')[0] == q['jawapan']

In [44]:
(correct / len(filtered)) * 100

28.846153846153843